In [ ]:
class Solution:

    def lengthOfLIS(self, nums: List[int], k: int) -> int:
        # dp[i] 以 i数 结尾的LIS
        # dp[i] = max(dp[j])  i - k <= j <= i - 1  从前面转移
        # basecase dp[mn] = 1
        mn, mx = min(nums), max(nums)
        vals = [0] * (4 * (mx - mn + 1))

        def update(o, l, r, idx, val):  # idx = val
            if l == r:
                vals[o] = val
                return
            m = l + r >> 1
            if m >= idx: update(o * 2, l, m, idx, val)
            else: update(o * 2 + 1, m + 1, r, idx, val)
            vals[o] = max(vals[o * 2], vals[o * 2 + 1])

        def query(o, l, r, L, R):  # [L,R]
            if L <= l and r <= R:
                return vals[o]
            m = l + r >> 1
            ans = 0
            if m >= L: ans = max(ans, query(o * 2, l, m, L, R))
            if m + 1 <= R: ans = max(ans, query(o * 2 + 1, m + 1, r, L, R))
            return ans

        for x in nums:
            if x == mn:
                update(1, mn, mx, x, 1)
            else:
                update(1, mn, mx, x, query(1, mn, mx, max(mn, x - k), x - 1) + 1)
        return vals[1]

In [1]:
# 超时
from typing import List


class Solution:

    def lengthOfLIS(self, nums: List[int], k: int) -> int:
        n = len(nums)

        # @lru_cache(None)
        # def dfs(i): # 以nums[i]结尾的最长LIS
        #     ans = 1
        #     x = nums[i]
        #     for j in range(i):
        #         if x - k <= nums[j] < x:
        #             ans = max(ans, dfs(j) + 1)
        #     return ans

        # return max(dfs(i) for i in range(n))

        f = [1] * n
        for i, x in enumerate(nums):
            for j in range(i):
                if x - k <= nums[j] < x and f[j] + 1 > f[i]:
                    f[i] = f[j] + 1
        return max(f)

In [ ]:
from typing import List


class Solution:

    def lengthOfLIS(self, nums: List[int], k: int) -> int:
        # n = len(nums)
        # mn, mx = min(nums), max(nums)

        # d[i][j] 前i个元素以j结尾的LIS
        # if nums[i]!=j: dp[i-1][j]
        # else: max(dp[i-1][j'])+1  j-k<=j'<j
        # i:[0,n] j:[mn,mx]
        # f[j] = max(f[j']+1) j-k<=j'<j x==j # 倒序

        # @lru_cache(None)
        # def dfs(i, j):
        #     if i < 0:
        #         return 0
        #     if nums[i] != j:
        #         return dfs(i - 1, j)
        #     ans = 1
        #     for x in range(max(1, j - k), j):
        #         if (sub := dfs(i - 1, x) + 1) > ans:
        #             ans = sub
        #     return ans

        # return max(dfs(n - 1, i) for i in range(mn, mx + 1))

        # f = [[0] * (mx + 1) for _ in range(n + 1)]
        # for i, num in enumerate(nums):
        #     for j in range(mn, mx + 1):
        #         if num != j:
        #             f[i + 1][j] = f[i][j]
        #         else:
        #             f[i + 1][j] = 1
        #             for x in range(max(1, j - k), j):
        #                 if f[i][x] + 1 > f[i + 1][j]:
        #                     f[i + 1][j] = f[i][x] + 1
        # return max(f[n])

        # f = [[0] * (mx + 1) for _ in range(2)]
        # for i, num in enumerate(nums):
        #     for j in range(mn, mx + 1):
        #         if num != j:
        #             f[i + 1 & 1][j] = f[i & 1][j]
        #         else:
        #             f[i + 1 & 1][j] = 1
        #             for x in range(max(1, j - k), j):
        #                 if f[i & 1][x] + 1 > f[i + 1 & 1][j]:
        #                     f[i + 1 & 1][j] = f[i & 1][x] + 1
        # return max(f[n & 1])

        # f = [0] * (mx + 1)
        # for num in nums:
        #     for j in range(mx, mn - 1, -1):
        #         if num == j:
        #             f[j] = 1
        #             for x in range(max(1, j - k), j): # 区间最大值
        #                 if f[x] + 1 > f[j]:
        #                     f[j] = f[x] + 1
        # return max(f)

        # f = [0] * (mx + 1)
        # for num in nums:
        #     for j in range(mx, mn - 1, -1):
        #         if num == j:
        #             f[j] = 1
        #             if j - 1 >= (left:=max(1, j - k)):
        #                 f[j] = max(f[left:j]) + 1
        # return max(f)


        # f = [0] * (mx + 1)
        # for x in nums:
        #     f[x] = 1
        #     if x - 1 >= (left:=max(1, x - k)):
        #         f[x] = max(f[left: x]) + 1
        # return max(f)

        u = max(nums) + 1
        mxo = [0] * (2 << u.bit_length())
        lazy = [None] * (2 << u.bit_length())

        def pushdown(o):
            if lazy[o] is not None:
                mxo[o * 2] = mxo[o * 2 + 1] = lazy[o * 2] = lazy[o * 2 + 1] = lazy[o]
                lazy[o] = None

        def update(o, l, r, idx, val): # 单点设置
            if l == r:
                mxo[o] = val
                return
            pushdown(o)
            m = l + r >> 1
            if m >= idx:
                update(o * 2, l, m, idx, val)
            else:
                update(o * 2 + 1, m + 1, r, idx, val)
            mxo[o] = max(mxo[o * 2], mxo[o * 2 + 1])

        def query(o, l, r, L, R): # 区间查询最大值
            if L <= l and r <= R:
                return mxo[o]
            pushdown(o)
            m = l + r >> 1
            ans = 0
            if m >= L:
                ans = query(o * 2, l, m, L, R)
            if m + 1 <= R:
                ans = max(ans, query(o * 2 + 1, m + 1, r, L, R))
            return ans

        # for x in nums:
        #     if x - 1 >= (left:=max(1, x - k)):
        #         update(1, 0, u - 1, x, query(1, 0, u - 1, left, x - 1) + 1)
        #     else:
        #         update(1, 0, u - 1, x, 1)
        # return mxo[1]

        for x in nums:
            if x != 1:
                update(1, 0, u - 1, x, query(1, 0, u - 1, max(1, x - k), x - 1) + 1)
            else:
                update(1, 0, u - 1, x, 1)
        return mxo[1]